In [103]:
import glob
import numpy

def get_file_header_info(file):
    with open(file, 'r') as f:
        lines = f.readlines()
    lines  = numpy.array([ l.replace('\n','').strip() for l in lines],dtype=str)
    return dict([ [ e.strip() for e in l.split(':',1) ] for l in lines[slice(*(numpy.logical_or(lines=='---\n',lines=='---').nonzero()[0] + (1,0)))] ])
def get_md_file_header_tag(file_or_file_header_info,tag):
    if type(file_or_file_header_info) is list:
        return [ get_md_file_header_tag(f,tag) for f in file_or_file_header_info ]
    else:
        h = file_or_file_header_info if type(file_or_file_header_info) is dict else get_file_header_info(file_or_file_header_info)
        return h[tag] if tag in h.keys() else ''
def find_file_by_tag(finfo_list,tag,value):
    return [ f for f in finfo_list if (value in f[tag]) ]
def sort_files_by_tag(finfo_list,tag,value_to_index_func=int,reverse=False):
    return sorted(finfo_list,key=lambda fi: value_to_index_func(fi[tag]),reverse=reverse)
def get_unique_values_for_tag(finfo_list,tag,dtype=None):
    return numpy.unique(numpy.array(get_md_file_header_tag(finfo_list,tag),dtype=dtype))
def sort_elements(arr,order,reverse=False):
    order_map = {key: i for i, key in enumerate(order)}
    return sorted(arr,key=lambda d: order_map[d],reverse=reverse)

In [104]:
all_files        = glob.glob('./_publications/*.md')
pubtype_tag      = 'pubtype'
year_tag         = 'year'
pubtype_order    = ['paper','book','proceedings','thesis']
pubtype_link_txt = dict(paper='Peer-reviewed papers',book='Books',proceedings='Proceedings',thesis='Theses')

finfo = []
for file in all_files:
    finfo.append(get_file_header_info(file))

In [108]:
def get_toc_li(href,id,txt,closetag=False):
    return r'<li><a href="#'+href+'" id="markdown-toc-'+id+'">'+txt+'</a>' + ('</li>' if closetag else '')

html_ul_code          = r'<ul class="toc__menu" id="markdown-toc">'
all_pubtype           = sort_elements(get_unique_values_for_tag(finfo,pubtype_tag),pubtype_order)
for ptype in all_pubtype:
    html_ul_code     += get_toc_li('toc-'+ptype,ptype,pubtype_link_txt[ptype],False)
    html_ul_code     += r'<ul>'
    all_years         = numpy.flip(get_unique_values_for_tag(find_file_by_tag(finfo,pubtype_tag,ptype),year_tag,dtype=int))
    for year in all_years:
        html_ul_code += get_toc_li('toc-'+ptype+'-'+str(year),ptype+'-'+str(year),str(year),True)
    html_ul_code     += r'</ul></li>' # closing year list for each ptype

html_ul_code         += r'</ul>'

html_toc_code = r'<nav class="toc sticky" style="height:200px;overflow:scroll;"><header><h4 class="nav__title"><i class="fa fa-file-text"></i>Skip to...</h4></header>'+html_ul_code+'</nav>'

print(html_toc_code)

outfilename = '_includes/toc-publications'
with open(outfilename,'w') as outfile:
    outfile.write(html_toc_code)
print('File written ... ',outfilename)

<nav class="toc sticky" style="height:200px;overflow:scroll;"><header><h4 class="nav__title"><i class="fa fa-file-text"></i>Skip to...</h4></header><ul class="toc__menu" id="markdown-toc"><li><a href="#toc-paper" id="markdown-toc-paper">Peer-reviewed papers</a><ul><li><a href="#toc-paper-2022" id="markdown-toc-paper-2022">2022</a></li><li><a href="#toc-paper-2021" id="markdown-toc-paper-2021">2021</a></li><li><a href="#toc-paper-2020" id="markdown-toc-paper-2020">2020</a></li><li><a href="#toc-paper-2019" id="markdown-toc-paper-2019">2019</a></li><li><a href="#toc-paper-2018" id="markdown-toc-paper-2018">2018</a></li><li><a href="#toc-paper-2017" id="markdown-toc-paper-2017">2017</a></li><li><a href="#toc-paper-2016" id="markdown-toc-paper-2016">2016</a></li><li><a href="#toc-paper-2013" id="markdown-toc-paper-2013">2013</a></li><li><a href="#toc-paper-2010" id="markdown-toc-paper-2010">2010</a></li></ul></li><li><a href="#toc-book" id="markdown-toc-book">Books</a><ul><li><a href="#toc